# Predicting Click rates

## You are given models that should predict if a customer will click on an ad as well as corresponding data. For testing purposes the models have all the same interfact and use the numeric data as input. The models are build with XGBoost version 3.0.5.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

In [ ]:
df = pd.read_csv('click_prediction.csv')